In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import warnings as wn
import sklearn.preprocessing as skpre
import category_encoders as ce

wn.simplefilter( "ignore" )

In [2]:
loc_ftr = r"D:\TP2_Machine_Learning_v4\FeaturesKaggle\FeaturesSC"

## <span style="color:yellow"> **Preparamos el set de entramiento para sacar features** </span> 

In [4]:
loc_ts = r"D:\TP2_Machine_Learning_v4\Training Sets"

In [5]:
evt_trn = pd.read_csv( loc_ts + "\\evt_21_23.csv" ); 
evt_tst = pd.read_csv( loc_ts + "\\evt_24_26.csv" );  

## <span style="color:yellow"> **Preparamos el ref_hash de cada ventana** </span> 

In [6]:
loc_lb = r"D:\TP2_Machine_Learning_v4\Labels"

In [7]:
lb_ins_trn = pd.read_csv( loc_lb + "\\label_ins_24_26.csv" );
lb_ins_tst = pd.read_csv( loc_lb + "\\target.csv" ); 

In [8]:
rh_trn = lb_ins_trn[ ["ref_hash"] ]
rh_tst = lb_ins_tst[ ["ref_hash"] ]

## <span style="color:yellow"> **=======================================================================================================** </span> 

## <span style="color:green"> **Horario del primer evento en ese ventana** </span> 

Al agregarle la transformacion senoidal para agregarle perdiodicidad a la hora, la prediccion del algoritmo fue peor por lo que decidimos no agregarla

In [9]:
hr_f_evt_trn = rh_trn.copy()
hr_f_evt_tst = rh_tst.copy()

first_evt_hour_trn = evt_trn[ ["ref_hash","date"] ].sort_values( "date" ).drop_duplicates( subset = "ref_hash", keep = "first" )
first_evt_hour_tst = evt_tst[ ["ref_hash","date"] ].sort_values( "date" ).drop_duplicates( subset = "ref_hash", keep = "first" )

first_evt_hour_trn["time_to_frt_evt"] = ( pd.to_datetime( first_evt_hour_trn["date"] ) -  dt.datetime( year = 2019, month = 4, day = 18 ) ).dt.total_seconds()
first_evt_hour_tst["time_to_frt_evt"] = ( pd.to_datetime( first_evt_hour_tst["date"] ) -  dt.datetime( year = 2019, month = 4, day = 21 ) ).dt.total_seconds()

first_evt_hour_trn["hour_frt_evt"] = pd.to_datetime( first_evt_hour_trn["date"] ).dt.hour
first_evt_hour_tst["hour_frt_evt"] = pd.to_datetime( first_evt_hour_tst["date"] ).dt.hour

first_evt_hour_trn.drop( ["date", "time_to_frt_evt"], axis = 1, inplace = True )
first_evt_hour_tst.drop( ["date", "time_to_frt_evt"], axis = 1, inplace = True )

hr_f_evt_trn = hr_f_evt_trn.merge( first_evt_hour_trn, how = "left", on = "ref_hash" )
hr_f_evt_tst = hr_f_evt_tst.merge( first_evt_hour_tst, how = "left", on = "ref_hash" )

#hr_f_evt_trn["hour_frt_evt"] = hr_f_evt_trn["hour_frt_evt"].apply( lambda x: np.sin( (x *np.pi)/24 ) )
#hr_f_evt_tst["hour_frt_evt"] = hr_f_evt_tst["hour_frt_evt"].apply( lambda x: np.sin( (x *np.pi)/24 ) )

hr_f_evt_trn.to_csv( loc_ftr + "\\hr_f_evt_trn.csv", index = False )
hr_f_evt_tst.to_csv( loc_ftr + "\\hr_f_evt_tst.csv", index = False )

Si bien hay varios valores NaN en el feature, lo decidimos probar de igual manera, ya que los algoritmos de boosting aceptan valores NaNs.

* https://datascience.stackexchange.com/questions/15305/how-does-xgboost-learn-what-are-the-inputs-for-missing-values/15306#15306
* The procedure is described in [their paper, section 3.4: Sparsity aware split-finding](https://arxiv.org/pdf/1603.02754v3.pdf).


## <span style="color:green"> **Cantidad de Eventos por dispositivo en la ventana previa a la conversion** </span>

In [10]:
cant_evt_trn = rh_trn.copy()
cant_evt_tst = rh_tst.copy()

cant_evt_trn = cant_evt_trn.merge( evt_trn["ref_hash"].value_counts().to_frame().reset_index().rename( columns = {"ref_hash": "cant_evt", "index":"ref_hash"} ), how = "left", on = "ref_hash" )
cant_evt_tst = cant_evt_tst.merge( evt_tst["ref_hash"].value_counts().to_frame().reset_index().rename( columns = {"ref_hash": "cant_evt", "index":"ref_hash"} ), how = "left", on = "ref_hash" )

cant_evt_trn.fillna( 0, inplace = True )
cant_evt_tst.fillna( 0, inplace = True )

cant_evt_trn.to_csv( loc_ftr + "\\cant_evt_trn.csv", index = False )
cant_evt_tst.to_csv( loc_ftr + "\\cant_evt_tst.csv", index = False )

## <span style="color:green"> **Cantidad de eventos atribuidas por dispositivo** </span> 

In [11]:
cevt_atr_trn = rh_trn.copy()
cevt_atr_tst = rh_tst.copy()

cant_atr_evt_trn = evt_trn[["ref_hash", "attributed"]]; cant_atr_evt_trn["attributed"] = cant_atr_evt_trn["attributed"].apply( lambda x: 1 if x else 0 );
cant_atr_evt_tst = evt_tst[["ref_hash", "attributed"]]; cant_atr_evt_tst["attributed"] = cant_atr_evt_tst["attributed"].apply( lambda x: 1 if x else 0 );

cant_atr_evt_trn = cant_atr_evt_trn.groupby( "ref_hash" ).agg( "sum" )
cant_atr_evt_tst = cant_atr_evt_tst.groupby( "ref_hash" ).agg( "sum" )

cevt_atr_trn = cevt_atr_trn.merge( cant_atr_evt_trn, how = "left", on = "ref_hash" ).fillna( 0 )
cevt_atr_tst = cevt_atr_tst.merge( cant_atr_evt_tst, how = "left", on = "ref_hash" ).fillna( 0 )

cevt_atr_trn.to_csv( loc_ftr + "\\cevt_atr_trn.csv", index = False )
cevt_atr_tst.to_csv( loc_ftr + "\\cevt_atr_tst.csv", index = False )

## <span style="color:green"> **Tiempo hasta el primer evento en ese ventana** </span> 

Le asignamos cuanto tiempo, en la ventana del 18-20, tardo en realizar su primer evento

In [12]:
frst_evt_trn = rh_trn.copy()
frst_evt_tst = rh_tst.copy()

first_event_rh_ins_trn = evt_trn[ ["ref_hash","date"] ].sort_values( "date" ).drop_duplicates( subset = "ref_hash", keep = "first" )
first_event_rh_ins_tst = evt_tst[ ["ref_hash","date"] ].sort_values( "date" ).drop_duplicates( subset = "ref_hash", keep = "first" )

first_event_rh_ins_trn["time_to_frt_evt"] = ( pd.to_datetime( first_event_rh_ins_trn["date"] ) -  dt.datetime( year = 2019, month = 4, day = 18 ) ).dt.total_seconds()
first_event_rh_ins_tst["time_to_frt_evt"] = ( pd.to_datetime( first_event_rh_ins_tst["date"] ) -  dt.datetime( year = 2019, month = 4, day = 21 ) ).dt.total_seconds()

first_event_rh_ins_trn.drop( "date", axis = 1, inplace = True )
first_event_rh_ins_tst.drop( "date", axis = 1, inplace = True )

frst_evt_trn = frst_evt_trn.merge( first_event_rh_ins_trn, how = "left", on = "ref_hash" )
frst_evt_tst = frst_evt_tst.merge( first_event_rh_ins_tst, how = "left", on = "ref_hash" )

#Los que tienen NaN es que nunca convirtieron. Los marcamos con el tiempo maximo
frst_evt_trn.fillna( 3 * 24 * 3600, inplace = True )
frst_evt_tst.fillna( 3 * 24 * 3600, inplace = True )

frst_evt_trn.to_csv( loc_ftr + "\\frst_evt_trn.csv", index = False )
frst_evt_tst.to_csv( loc_ftr + "\\frst_evt_tst.csv", index = False )

## <span style="color:green"> **Tipo de evento mas realizado por el dispositivo** </span> 

#### <span style="color:orange"> **Mean Encoding** </span> (Lo codificamos haciendo la cantidad de eventos que se realizaron de ese tipo sobre el total de eventos realizados)

In [13]:
kind_evt_trn = rh_trn.copy()
kind_evt_tst = rh_tst.copy()

main_kind_evt_trn = evt_trn.groupby( by = ["ref_hash","kind"] ).agg( {"kind":"count"} ).rename( columns = {"kind":"cant_evt"} ).reset_index()
main_kind_evt_tst = evt_tst.groupby( by = ["ref_hash","kind"] ).agg( {"kind":"count"} ).rename( columns = {"kind":"cant_evt"} ).reset_index()

main_kind_evt_trn = main_kind_evt_trn.sort_values( by = ["ref_hash", "cant_evt"], ascending = True ).drop_duplicates( subset = "ref_hash", keep = "last" ); del( main_kind_evt_trn["cant_evt"] )
main_kind_evt_tst = main_kind_evt_tst.sort_values( by = ["ref_hash", "cant_evt"], ascending = True ).drop_duplicates( subset = "ref_hash", keep = "last" ); del( main_kind_evt_tst["cant_evt"] ) 

kind_evt_trn = kind_evt_trn.merge( main_kind_evt_trn, how = "left", on = "ref_hash" )
kind_evt_tst = kind_evt_tst.merge( main_kind_evt_tst, how = "left", on = "ref_hash" )

cant_kind_evt_trn = evt_trn[["ref_hash","kind"]].groupby( "kind" ).agg("count").reset_index().rename( columns = {"ref_hash":"cant_kind"} )
cant_kind_evt_tst = evt_tst[["ref_hash","kind"]].groupby( "kind" ).agg("count").reset_index().rename( columns = {"ref_hash":"cant_kind"} )

kind_evt_trn = kind_evt_trn.merge( cant_kind_evt_trn, how = "left", on = "kind" ).drop( "kind", axis = 1 )
kind_evt_tst = kind_evt_tst.merge( cant_kind_evt_tst, how = "left", on = "kind" ).drop( "kind", axis = 1 )

kind_evt_trn["kind_mean"] = kind_evt_trn["cant_kind"] / len(evt_trn); kind_evt_trn.fillna( kind_evt_trn["kind_mean"].isnull().sum() / len(evt_trn), inplace = True ); del(kind_evt_trn["cant_kind"])
kind_evt_tst["kind_mean"] = kind_evt_tst["cant_kind"] / len(evt_tst); kind_evt_tst.fillna( kind_evt_tst["kind_mean"].isnull().sum() / len(evt_tst), inplace = True ); del(kind_evt_tst["cant_kind"])

kind_evt_trn.to_csv( loc_ftr + "\\kind_evt_trn.csv", index = False )
kind_evt_tst.to_csv( loc_ftr + "\\kind_evt_tst.csv", index = False )

## <span style="color:green"> **Aplicaciones que mas realizaron eventos cada usuario** </span>

#### <span style="color:orange"> **Mean Encoding** </span> (Usamos el promedio de la cantidad de veces que que es la app principal de algun dispositivo)

In [14]:
mapp_evt_trn = rh_trn.copy()
mapp_evt_tst = rh_tst.copy()

app_mas_evt_trn = evt_trn.groupby( by = ["ref_hash","application_id"] ).agg( {"application_id":"count"} ).rename( columns = {"application_id":"cant_evt"} ).reset_index()
app_mas_evt_tst = evt_tst.groupby( by = ["ref_hash","application_id"] ).agg( {"application_id":"count"} ).rename( columns = {"application_id":"cant_evt"} ).reset_index()

app_mas_evt_trn = app_mas_evt_trn.sort_values( by = ["ref_hash", "cant_evt"], ascending = True ).drop_duplicates( subset = "ref_hash", keep = "last" ); del( app_mas_evt_trn["cant_evt"] )
app_mas_evt_tst = app_mas_evt_tst.sort_values( by = ["ref_hash", "cant_evt"], ascending = True ).drop_duplicates( subset = "ref_hash", keep = "last" ); del( app_mas_evt_tst["cant_evt"] ) 

mapp_evt_trn = mapp_evt_trn.merge( app_mas_evt_trn, how = "left", on = "ref_hash" )
mapp_evt_tst = mapp_evt_tst.merge( app_mas_evt_tst, how = "left", on = "ref_hash" )

mapp_evt_trn["to_count"] = 1; mapp_evt_trn["application_id"] = mapp_evt_trn[["application_id", "to_count"]].groupby("application_id").transform( "sum" ) / len(mapp_evt_trn); del(mapp_evt_trn["to_count"])
mapp_evt_tst["to_count"] = 1; mapp_evt_tst["application_id"] = mapp_evt_tst[["application_id", "to_count"]].groupby("application_id").transform( "sum" ) / len(mapp_evt_tst); del(mapp_evt_tst["to_count"])

#Completamos los NaNs con el promedio de su cantidad
mapp_evt_trn["application_id"].fillna( mapp_evt_trn["application_id"].isnull().sum() / len( mapp_evt_trn ) , inplace = True )
mapp_evt_tst["application_id"].fillna( mapp_evt_tst["application_id"].isnull().sum() / len( mapp_evt_tst ), inplace = True )

mapp_evt_trn.to_csv( loc_ftr + "\\mapp_evt_trn.csv", index = False )
mapp_evt_tst.to_csv( loc_ftr + "\\mapp_evt_tst.csv", index = False )

## <span style="color:green"> **Cantidad de eventos realizados con WiFi** </span>  

In [15]:
wifi_evt_trn = rh_trn.copy()
wifi_evt_tst = rh_tst.copy()

cant_wifi_evt_trn = evt_trn[["ref_hash", "wifi"]]; cant_wifi_evt_trn["wifi"] = cant_wifi_evt_trn["wifi"].apply( lambda x: 1 if x else 0 );
cant_wifi_evt_tst = evt_tst[["ref_hash", "wifi"]]; cant_wifi_evt_tst["wifi"] = cant_wifi_evt_tst["wifi"].apply( lambda x: 1 if x else 0 );

cant_wifi_evt_trn = cant_wifi_evt_trn.groupby( "ref_hash" ).agg( "sum" )
cant_wifi_evt_tst = cant_wifi_evt_tst.groupby( "ref_hash" ).agg( "sum" )

wifi_evt_trn = wifi_evt_trn.merge( cant_wifi_evt_trn, how = "left", on = "ref_hash" ).fillna( 0 )
wifi_evt_tst = wifi_evt_tst.merge( cant_wifi_evt_tst, how = "left", on = "ref_hash" ).fillna( 0 )

wifi_evt_trn.to_csv( loc_ftr + "\\wifi_evt_trn.csv", index = False )
wifi_evt_tst.to_csv( loc_ftr + "\\wifi_evt_tst.csv", index = False )

## <span style="color:green"> **Tiempo hasta la ultimo evento en esa ventana** </span> 

In [16]:
last_evt_trn = rh_trn.copy()
last_evt_tst = rh_tst.copy()

last_event_rh_ins_trn = evt_trn[ ["ref_hash","date"] ].sort_values( "date", ascending = False ).drop_duplicates( subset = "ref_hash", keep = "first" )
last_event_rh_ins_tst = evt_tst[ ["ref_hash","date"] ].sort_values( "date", ascending = False ).drop_duplicates( subset = "ref_hash", keep = "first" )

last_event_rh_ins_trn["time_to_lst_evt"] = ( pd.to_datetime( last_event_rh_ins_trn["date"] ) -  dt.datetime( year = 2019, month = 4, day = 18 ) ).dt.total_seconds()
last_event_rh_ins_tst["time_to_lst_evt"] = ( pd.to_datetime( last_event_rh_ins_tst["date"] ) -  dt.datetime( year = 2019, month = 4, day = 21 ) ).dt.total_seconds()

last_event_rh_ins_trn.drop( "date", axis = 1, inplace = True )
last_event_rh_ins_tst.drop( "date", axis = 1, inplace = True )

last_evt_trn = last_evt_trn.merge( last_event_rh_ins_trn, how = "left", on = "ref_hash" )
last_evt_tst = last_evt_tst.merge( last_event_rh_ins_tst, how = "left", on = "ref_hash" )

#Los que tienen NaN es que nunca convirtieron. Los marcamos con el tiempo maximo
last_evt_trn.fillna( 3 * 24 * 3600, inplace = True )
last_evt_tst.fillna( 3 * 24 * 3600, inplace = True )

last_evt_trn.to_csv( loc_ftr + "\\last_evt_trn.csv", index = False )
last_evt_tst.to_csv( loc_ftr + "\\last_evt_tst.csv", index = False )

## <span style="color:green"> **Hizo eventos entre 21 hs y 3 hs (Noche)** </span>

#### <span style="color:Orange"> **One-Hot Encoding** </span> 

In [19]:
evt_trn['date'] = pd.to_datetime(evt_trn['date'])
evt_tst['date'] = pd.to_datetime(evt_tst['date'])

evt_trn['evt_21_3'] = (evt_trn['date'].dt.hour < 4) | (evt_trn['date'].dt.hour > 20)
evt_tst['evt_21_3'] = (evt_tst['date'].dt.hour < 4) | (evt_tst['date'].dt.hour > 20)

evt_night_trn = rh_trn.copy()
evt_night_tst = rh_tst.copy()

hour_mode_trn = evt_trn.groupby('ref_hash').agg({'evt_21_3':'sum'}).reset_index()
hour_mode_tst = evt_tst.groupby('ref_hash').agg({'evt_21_3':'sum'}).reset_index()

evt_night_trn = evt_night_trn.merge( hour_mode_trn, how = "left", on = "ref_hash" )
evt_night_tst = evt_night_tst.merge( hour_mode_tst, how = "left", on = "ref_hash" )

evt_night_trn["evt_21_3"] =  (evt_night_trn["evt_21_3"] > 1).astype('int8')
evt_night_tst["evt_21_3"] =  (evt_night_tst["evt_21_3"] > 1).astype('int8')

evt_night_trn.to_csv( loc_ftr + "\\evt_nght_trn.csv", index = False )
evt_night_tst.to_csv( loc_ftr + "\\evt_nght_tst.csv", index = False )

del evt_trn['evt_21_3']
del evt_tst['evt_21_3']

## <span style="color:green"> **Hizo eventos entre 4 hs y 10 hs (Mañana)** </span>

#### <span style="color:Orange"> **One-Hot Encoding** </span> 

In [20]:
evt_trn['date'] = pd.to_datetime(evt_trn['date'])
evt_tst['date'] = pd.to_datetime(evt_tst['date'])

evt_trn['evt_4_10'] = (evt_trn['date'].dt.hour < 11) & (evt_trn['date'].dt.hour > 3)
evt_tst['evt_4_10'] = (evt_tst['date'].dt.hour < 11) & (evt_tst['date'].dt.hour > 3)

evt_morn_trn = rh_trn.copy()
evt_morn_tst = rh_tst.copy()

hour_mode_trn = evt_trn.groupby('ref_hash').agg({'evt_4_10':'sum'}).reset_index()
hour_mode_tst = evt_tst.groupby('ref_hash').agg({'evt_4_10':'sum'}).reset_index()

evt_morn_trn = evt_morn_trn.merge( hour_mode_trn, how = "left", on = "ref_hash" )
evt_morn_tst = evt_morn_tst.merge( hour_mode_tst, how = "left", on = "ref_hash" )

evt_morn_trn["evt_4_10"] =  (evt_morn_trn["evt_4_10"] > 1).astype('int8')
evt_morn_tst["evt_4_10"] =  (evt_morn_tst["evt_4_10"] > 1).astype('int8')

evt_morn_trn.to_csv( loc_ftr + "\\evt_morn_trn.csv", index = False )
evt_morn_tst.to_csv( loc_ftr + "\\evt_morn_tst.csv", index = False )

del evt_trn['evt_4_10']
del evt_tst['evt_4_10']

## <span style="color:green"> **Hizo eventos entre 11 hs y 15 hs (Mediodia)** </span>

#### <span style="color:Orange"> **One-Hot Encoding** </span> 

In [22]:
evt_trn['date'] = pd.to_datetime(evt_trn['date'])
evt_tst['date'] = pd.to_datetime(evt_tst['date'])

evt_trn['evt_11_15'] = (evt_trn['date'].dt.hour < 16) & (evt_trn['date'].dt.hour > 10)
evt_tst['evt_11_15'] = (evt_tst['date'].dt.hour < 16) & (evt_tst['date'].dt.hour > 10)

evt_midday_trn = rh_trn.copy()
evt_midday_tst = rh_tst.copy()

hour_mode_trn = evt_trn.groupby('ref_hash').agg({'evt_11_15':'sum'}).reset_index()
hour_mode_tst = evt_tst.groupby('ref_hash').agg({'evt_11_15':'sum'}).reset_index()

evt_midday_trn = evt_midday_trn.merge( hour_mode_trn, how = "left", on = "ref_hash" )
evt_midday_tst = evt_midday_tst.merge( hour_mode_tst, how = "left", on = "ref_hash" )

evt_midday_trn["evt_11_15"] =  (evt_midday_trn["evt_11_15"] > 1).astype('int8')
evt_midday_tst["evt_11_15"] =  (evt_midday_tst["evt_11_15"] > 1).astype('int8')

evt_midday_trn.to_csv( loc_ftr + "\\evt_mday_tst.csv", index = False )
evt_midday_tst.to_csv( loc_ftr + "\\evt_mday_trn.csv", index = False )

del evt_trn['evt_11_15']
del evt_tst['evt_11_15']

## <span style="color:green"> **Hizo eventos entre 16 hs y 20 hs (Tarde)** </span>

#### <span style="color:Orange"> **One-Hot Encoding** </span> 

In [24]:
evt_trn['date'] = pd.to_datetime(evt_trn['date'])
evt_tst['date'] = pd.to_datetime(evt_tst['date'])

evt_trn['evt_16_20'] = (evt_trn['date'].dt.hour < 21) & (evt_trn['date'].dt.hour > 15)
evt_tst['evt_16_20'] = (evt_tst['date'].dt.hour < 21) & (evt_tst['date'].dt.hour > 15)

evt_after_trn = rh_trn.copy()
evt_after_tst = rh_tst.copy()

hour_mode_trn = evt_trn.groupby('ref_hash').agg({'evt_16_20':'sum'}).reset_index()
hour_mode_tst = evt_tst.groupby('ref_hash').agg({'evt_16_20':'sum'}).reset_index()

evt_after_trn = evt_after_trn.merge( hour_mode_trn, how = "left", on = "ref_hash" )
evt_after_tst = evt_after_tst.merge( hour_mode_tst, how = "left", on = "ref_hash" )

evt_after_trn["evt_16_20"] =  (evt_after_trn["evt_16_20"] > 1).astype('int8')
evt_after_tst["evt_16_20"] =  (evt_after_tst["evt_16_20"] > 1).astype('int8')

evt_after_trn.to_csv( loc_ftr + "\\evt_aftr_trn.csv", index = False )
evt_after_tst.to_csv( loc_ftr + "\\evt_aftr_tst.csv", index = False )

del evt_trn['evt_16_20']
del evt_tst['evt_16_20']

## <span style="color:green"> **Tiempo hasta que el usuario realizo algun evento evento del TOP 3** </span>

In [25]:
tt_3_evt_trn = rh_trn.copy()
tt_3_evt_tst = rh_tst.copy()

cond_1_trn = evt_trn['kind'] == evt_trn['kind'].value_counts().head().index[0]; cond_1_tst = evt_tst['kind'] == evt_tst['kind'].value_counts().head().index[0];
cond_2_trn = evt_trn['kind'] == evt_trn['kind'].value_counts().head().index[1]; cond_2_tst = evt_tst['kind'] == evt_tst['kind'].value_counts().head().index[1];
cond_3_trn = evt_trn['kind'] == evt_trn['kind'].value_counts().head().index[2]; cond_3_tst = evt_tst['kind'] == evt_tst['kind'].value_counts().head().index[2];

k1 = evt_trn.loc[ cond_1_trn | cond_2_trn | cond_3_trn, ["ref_hash", "date"] ].sort_values( by = "date", ascending = True ).drop_duplicates( subset = ["ref_hash"], keep = "first" )
k2 = evt_tst.loc[ cond_1_tst | cond_2_tst | cond_3_tst, ["ref_hash", "date"] ].sort_values( by = "date", ascending = True ).drop_duplicates( subset = ["ref_hash"], keep = "first" )

tt_3_evt_trn = tt_3_evt_trn.merge( k1, how = "left", on = "ref_hash" )
tt_3_evt_tst = tt_3_evt_tst.merge( k2, how = "left", on = "ref_hash" )

tt_3_evt_trn["date"] = ( pd.to_datetime( tt_3_evt_trn["date"] ) - dt.datetime( 2019, 4, 18 ) ).dt.total_seconds()
tt_3_evt_tst["date"] = ( pd.to_datetime( tt_3_evt_tst["date"] ) - dt.datetime( 2019, 4, 21 ) ).dt.total_seconds()

tt_3_evt_trn.fillna( 3*24*3600, inplace = True ) #Quiere decir que no lo realizo
tt_3_evt_tst.fillna( 3*24*3600, inplace = True ) #Quiere decir que no lo realizo

tt_3_evt_trn.to_csv( loc_ftr + "\\tt_3_evt_trn.csv", index = False )
tt_3_evt_tst.to_csv( loc_ftr + "\\tt_3_evt_tst.csv", index = False )

## <span style="color:green"> **Realizo algun evento del TOP 3** </span>
Se corroboro inicialmente que los top3 tipos de eventos coinciden en las 3 ventanas

#### <span style="color:orange"> **OneHotEncoding** </span>

In [26]:
top3_evt_trn = rh_trn.copy()
top3_evt_tst = rh_tst.copy()

cond_1_trn = evt_trn['kind'] == evt_trn['kind'].value_counts().head().index[0]; cond_1_tst = evt_tst['kind'] == evt_tst['kind'].value_counts().head().index[0];
cond_2_trn = evt_trn['kind'] == evt_trn['kind'].value_counts().head().index[1]; cond_2_tst = evt_tst['kind'] == evt_tst['kind'].value_counts().head().index[1];
cond_3_trn = evt_trn['kind'] == evt_trn['kind'].value_counts().head().index[2]; cond_3_tst = evt_tst['kind'] == evt_tst['kind'].value_counts().head().index[2];

k1 = evt_trn.loc[ cond_1_trn | cond_2_trn | cond_3_trn, ["ref_hash", "kind"] ]; k1["kind"] = k1["kind"].astype( "category" )
k2 = evt_tst.loc[ cond_1_tst | cond_2_tst | cond_3_tst, ["ref_hash", "kind"] ]; k2["kind"] = k2["kind"].astype( "category" )

k1 = k1.groupby( "ref_hash" ).agg( {"kind":"value_counts"} ).unstack( level = 1 ).fillna( 0 )
k2 = k2.groupby( "ref_hash" ).agg( {"kind":"value_counts"} ).unstack( level = 1 ).fillna( 0 )

top3_evt_trn = top3_evt_trn.merge( k1, how = "left", on = "ref_hash" ).fillna( 0 ); top3_evt_trn.columns = ["ref_hash", "evt1", "evt2", "evt3" ]
top3_evt_tst = top3_evt_tst.merge( k2, how = "left", on = "ref_hash" ).fillna( 0 ); top3_evt_tst.columns = ["ref_hash", "evt1", "evt2", "evt3" ]

top3_evt_trn["evt1"] = top3_evt_trn["evt1"].apply( lambda x: 1 if x > 0 else 0 ); top3_evt_tst["evt1"] = top3_evt_tst["evt1"].apply( lambda x: 1 if x > 0 else 0 );
top3_evt_trn["evt2"] = top3_evt_trn["evt2"].apply( lambda x: 1 if x > 0 else 0 ); top3_evt_tst["evt2"] = top3_evt_tst["evt1"].apply( lambda x: 1 if x > 0 else 0 );
top3_evt_trn["evt3"] = top3_evt_trn["evt3"].apply( lambda x: 1 if x > 0 else 0 ); top3_evt_tst["evt3"] = top3_evt_tst["evt3"].apply( lambda x: 1 if x > 0 else 0 );

top3_evt_trn.to_csv( loc_ftr + "\\top3_evt_trn.csv", index = False )
top3_evt_tst.to_csv( loc_ftr + "\\top3_evt_tst.csv", index = False )

## <span style="color:green"> **Realizo un evento de la aplicacion: 210** </span>
Esta aplicacion es la mas popular respecto a eventos, con mas del 25% del total

In [27]:
evt_a210_trn = rh_trn.copy()
evt_a210_tst = rh_tst.copy()

do_evt_app_210_trn = evt_trn[ ["ref_hash", "application_id"] ]; do_evt_app_210_trn["do_it_210"] = do_evt_app_210_trn["application_id"].apply( lambda x: 1 if x == 210 else 0 )
do_evt_app_210_tst = evt_tst[ ["ref_hash", "application_id"] ]; do_evt_app_210_tst["do_it_210"] = do_evt_app_210_tst["application_id"].apply( lambda x: 1 if x == 210 else 0 )

do_evt_app_210_trn = do_evt_app_210_trn.groupby('ref_hash').agg({'do_it_210':'max'}).reset_index()
do_evt_app_210_tst = do_evt_app_210_tst.groupby('ref_hash').agg({'do_it_210':'max'}).reset_index()

evt_a210_trn = evt_a210_trn.merge( do_evt_app_210_trn, how = "left", on = "ref_hash" );
evt_a210_tst = evt_a210_tst.merge( do_evt_app_210_tst, how = "left", on = "ref_hash" );

evt_a210_trn.to_csv( loc_ftr + "\\evt_a210_trn.csv", index = False )
evt_a210_tst.to_csv( loc_ftr + "\\evt_a210_tst.csv", index = False )


## <span style="color:green"> **Realizo un evento de ID: 1** </span>
Esta id de evento es la mas popular en todas las ventanas

In [28]:
evt_id01_trn = rh_trn.copy()
evt_id01_tst = rh_tst.copy()

do_evt_id_1_trn = evt_trn[ ["ref_hash", "event_id"] ]; do_evt_id_1_trn["do_id_1"] = do_evt_id_1_trn["event_id"].apply( lambda x: 1 if x == 1 else 0 )
do_evt_id_1_tst = evt_tst[ ["ref_hash", "event_id"] ]; do_evt_id_1_tst["do_id_1"] = do_evt_id_1_tst["event_id"].apply( lambda x: 1 if x == 1 else 0 )

do_evt_id_1_trn = do_evt_id_1_trn.groupby('ref_hash').agg({'do_id_1':'max'}).reset_index()
do_evt_id_1_tst = do_evt_id_1_tst.groupby('ref_hash').agg({'do_id_1':'max'}).reset_index()

evt_id01_trn = evt_id01_trn.merge( do_evt_id_1_trn, how = "left", on = "ref_hash" );
evt_id01_tst = evt_id01_tst.merge( do_evt_id_1_tst, how = "left", on = "ref_hash" );

evt_a210_trn.to_csv( loc_ftr + "\\evt_id01_trn.csv", index = False )
evt_a210_tst.to_csv( loc_ftr + "\\evt_id01_tst.csv", index = False )

## <span style="color:yellow"> ============================================================================================================================ </span> 

## <span style="color:green"> **Tiene Wifi en primer evento** </span>

In [29]:
frst_evt_trn = rh_trn.copy()
frst_evt_tst = rh_tst.copy()

wifi_first_event_rh_ins_trn = evt_trn[ ["ref_hash","date",'wifi'] ].sort_values( "date" ).drop_duplicates( subset = "ref_hash", keep = "first" )
wifi_first_event_rh_ins_tst = evt_tst[ ["ref_hash","date",'wifi'] ].sort_values( "date" ).drop_duplicates( subset = "ref_hash", keep = "first" )

wifi_first_event_rh_ins_trn.drop( "date", axis = 1, inplace = True )
wifi_first_event_rh_ins_tst.drop( "date", axis = 1, inplace = True )

frst_evt_trn = frst_evt_trn.merge( wifi_first_event_rh_ins_trn, how = "left", on = "ref_hash" )
frst_evt_tst = frst_evt_tst.merge( wifi_first_event_rh_ins_tst, how = "left", on = "ref_hash" )

frst_evt_trn['wifi'] = (frst_evt_trn['wifi'] == True).astype('int8')
frst_evt_tst['wifi'] = (frst_evt_tst['wifi'] == True).astype('int8')

frst_evt_trn.to_csv( loc_ftr + "/wif_fevt_trn.csv", index = False )
frst_evt_tst.to_csv( loc_ftr + "/wif_fevt_tst.csv", index = False )

## <span style="color:green"> **Tiene Wifi en ultimo evento** </span>

In [30]:
last_evt_trn = rh_trn.copy()
last_evt_tst = rh_tst.copy()

wifi_last_event_rh_ins_trn = evt_trn[ ["ref_hash","date",'wifi'] ].sort_values( "date", ascending = False ).drop_duplicates( subset = "ref_hash", keep = "first" )
wifi_last_event_rh_ins_tst = evt_tst[ ["ref_hash","date",'wifi'] ].sort_values( "date", ascending = False ).drop_duplicates( subset = "ref_hash", keep = "first" )

wifi_last_event_rh_ins_trn.drop( "date", axis = 1, inplace = True )
wifi_last_event_rh_ins_tst.drop( "date", axis = 1, inplace = True )

last_evt_trn = last_evt_trn.merge( wifi_last_event_rh_ins_trn, how = "left", on = "ref_hash" )
last_evt_tst = last_evt_tst.merge( wifi_last_event_rh_ins_tst, how = "left", on = "ref_hash" )

last_evt_trn['wifi'] = (last_evt_trn['wifi'] == True).astype('int8')
last_evt_tst['wifi'] = (last_evt_tst['wifi'] == True).astype('int8')

last_evt_trn.to_csv( loc_ftr + "/wif_levt_trn.csv", index = False )
last_evt_tst.to_csv( loc_ftr + "/wif_levt_tst.csv", index = False )

## <span style="color:green"> **Tipo del ultimo evento realizado (kind)** </span>

#### <span style='color:orange'> **Mean encoding** </span>

In [31]:
kind_evt_trn = rh_trn.copy()
kind_evt_tst = rh_tst.copy()

kind_last_event_rh_ins_trn = evt_trn[ ["ref_hash","date",'kind'] ].sort_values( "date", ascending = False ).drop_duplicates( subset = "ref_hash", keep = "first" )
kind_last_event_rh_ins_tst = evt_tst[ ["ref_hash","date",'kind'] ].sort_values( "date", ascending = False ).drop_duplicates( subset = "ref_hash", keep = "first" )

kind_last_event_rh_ins_trn.drop( "date", axis = 1, inplace = True )
kind_last_event_rh_ins_tst.drop( "date", axis = 1, inplace = True )

kind_evt_trn = kind_evt_trn.merge( kind_last_event_rh_ins_trn, how = "left", on = "ref_hash" )
kind_evt_tst = kind_evt_tst.merge( kind_last_event_rh_ins_tst, how = "left", on = "ref_hash" )

kind_evt_trn['count_kind'] = kind_evt_trn.groupby('kind')['kind'].transform('count')
kind_evt_tst['count_kind'] = kind_evt_tst.groupby('kind')['kind'].transform('count')

kind_evt_trn["kind"] = kind_evt_trn["count_kind"] / len(kind_evt_trn); 
kind_evt_trn.fillna( kind_evt_trn["kind"].isnull().sum() / len(kind_evt_trn), inplace = True ); 
del(kind_evt_trn["count_kind"])

kind_evt_tst["kind"] = kind_evt_tst["count_kind"] / len(kind_evt_tst); 
kind_evt_tst.fillna( kind_evt_tst["kind"].isnull().sum() / len(kind_evt_tst), inplace = True ); 
del(kind_evt_tst["count_kind"])

kind_evt_trn.to_csv( loc_ftr + "/knd_levt_trn.csv", index = False )
kind_evt_tst.to_csv( loc_ftr + "/knd_levt_tst.csv", index = False )

## <span style="color:green"> **Cantidad de eventos entre 3 hs y 21 hs (Noche)** </span>

In [32]:
evt_trn['date'] = pd.to_datetime(evt_trn['date'])
evt_tst['date'] = pd.to_datetime(evt_tst['date'])

evt_trn['evt_21_3'] = (evt_trn['date'].dt.hour < 4) | (evt_trn['date'].dt.hour > 20)
evt_tst['evt_21_3'] = (evt_tst['date'].dt.hour < 4) | (evt_tst['date'].dt.hour > 20)

evt_night_trn = rh_trn.copy()
evt_night_tst = rh_tst.copy()

hour_mode_trn = evt_trn.groupby('ref_hash').agg({'evt_21_3':'sum'}).reset_index()
hour_mode_tst = evt_tst.groupby('ref_hash').agg({'evt_21_3':'sum'}).reset_index()

evt_night_trn = evt_night_trn.merge( hour_mode_trn, how = "left", on = "ref_hash" )
evt_night_tst = evt_night_tst.merge( hour_mode_tst, how = "left", on = "ref_hash" )

evt_night_trn.fillna( 0.0 , inplace = True )
evt_night_tst.fillna( 0.0 , inplace = True )

evt_night_trn.to_csv( loc_ftr + "\\cevt_nig_trn.csv", index = False )
evt_night_tst.to_csv( loc_ftr + "\\cevt_nig_tst.csv", index = False )

del evt_trn['evt_21_3']
del evt_tst['evt_21_3']


## <span style="color:green"> **Cantidad de eventos entre 4 hs y 10 hs (Mañana)** </span>

In [33]:
evt_trn['date'] = pd.to_datetime(evt_trn['date'])
evt_tst['date'] = pd.to_datetime(evt_tst['date'])

evt_trn['evt_4_10'] = (evt_trn['date'].dt.hour < 11) & (evt_trn['date'].dt.hour > 3)
evt_tst['evt_4_10'] = (evt_tst['date'].dt.hour < 11) & (evt_tst['date'].dt.hour > 3)

evt_morn_trn = rh_trn.copy()
evt_morn_tst = rh_tst.copy()

hour_mode_trn = evt_trn.groupby('ref_hash').agg({'evt_4_10':'sum'}).reset_index()
hour_mode_tst = evt_tst.groupby('ref_hash').agg({'evt_4_10':'sum'}).reset_index()

evt_morn_trn = evt_morn_trn.merge( hour_mode_trn, how = "left", on = "ref_hash" )
evt_morn_tst = evt_morn_tst.merge( hour_mode_tst, how = "left", on = "ref_hash" )

evt_morn_trn.fillna( 0.0 , inplace = True )
evt_morn_tst.fillna( 0.0 , inplace = True )

evt_morn_trn.to_csv( loc_ftr + "\\cevt_mor_trn.csv", index = False )
evt_morn_tst.to_csv( loc_ftr + "\\cevt_mor_tst.csv", index = False )

del evt_trn['evt_4_10']
del evt_tst['evt_4_10']


## <span style="color:green"> **Cantidad de eventos entre 11 hs y 15 hs (Mediodia)** </span>

In [35]:
evt_trn['date'] = pd.to_datetime(evt_trn['date'])
evt_tst['date'] = pd.to_datetime(evt_tst['date'])

evt_trn['evt_11_15'] = (evt_trn['date'].dt.hour < 16) & (evt_trn['date'].dt.hour > 10)
evt_tst['evt_11_15'] = (evt_tst['date'].dt.hour < 16) & (evt_tst['date'].dt.hour > 10)

evt_midday_trn = rh_trn.copy()
evt_midday_tst = rh_tst.copy()

hour_mode_trn = evt_trn.groupby('ref_hash').agg({'evt_11_15':'sum'}).reset_index()
hour_mode_tst = evt_tst.groupby('ref_hash').agg({'evt_11_15':'sum'}).reset_index()

evt_midday_trn = evt_midday_trn.merge( hour_mode_trn, how = "left", on = "ref_hash" )
evt_midday_tst = evt_midday_tst.merge( hour_mode_tst, how = "left", on = "ref_hash" )

evt_midday_trn.fillna( 0.0 , inplace = True )
evt_midday_tst.fillna( 0.0 , inplace = True )

evt_midday_trn.to_csv( loc_ftr + "\\cevt_mid_trn.csv", index = False )
evt_midday_tst.to_csv( loc_ftr + "\\cevt_mid_tst.csv", index = False )

del evt_trn['evt_11_15']
del evt_tst['evt_11_15']


## <span style="color:green"> **Cantidad de eventos entre 16 hs y 20 hs (Noche)** </span>

In [37]:
evt_trn['date'] = pd.to_datetime(evt_trn['date'])
evt_tst['date'] = pd.to_datetime(evt_tst['date'])

evt_trn['evt_16_20'] = (evt_trn['date'].dt.hour < 21) & (evt_trn['date'].dt.hour > 15)
evt_tst['evt_16_20'] = (evt_tst['date'].dt.hour < 21) & (evt_tst['date'].dt.hour > 15)

evt_after_18_20 = rh_trn.copy()
evt_after_tst = rh_tst.copy()

hour_mode_trn = evt_trn.groupby('ref_hash').agg({'evt_16_20':'sum'}).reset_index()
hour_mode_tst = evt_tst.groupby('ref_hash').agg({'evt_16_20':'sum'}).reset_index()

evt_after_trn = evt_after_trn.merge( hour_mode_trn, how = "left", on = "ref_hash" )
evt_after_tst = evt_after_tst.merge( hour_mode_tst, how = "left", on = "ref_hash" )

evt_after_trn.fillna( 0.0 , inplace = True )
evt_after_tst.fillna( 0.0 , inplace = True )

evt_after_trn.to_csv( loc_ftr + "\\cevt_aft_trn.csv", index = False )
evt_after_tst.to_csv( loc_ftr + "\\cevt_aft_tst.csv", index = False )

del evt_trn['evt_16_20']
del evt_tst['evt_16_20']


## <span style="color:green"> **Cantidad eventos atribuidos con wifi** </span>

In [38]:
cevt_atr_trn = rh_trn.copy()
cevt_atr_tst = rh_tst.copy()

cant_atr_evt_trn = evt_trn.loc[evt_trn['wifi'] == True, ["ref_hash", "attributed"]]; cant_atr_evt_trn["attributed"] = cant_atr_evt_trn["attributed"].apply( lambda x: 1 if x else 0 );
cant_atr_evt_tst = evt_tst.loc[evt_tst['wifi'] == True, ["ref_hash", "attributed"]]; cant_atr_evt_tst["attributed"] = cant_atr_evt_tst["attributed"].apply( lambda x: 1 if x else 0 );

cant_atr_evt_trn = cant_atr_evt_trn.groupby( "ref_hash" ).agg( "sum" )
cant_atr_evt_tst = cant_atr_evt_tst.groupby( "ref_hash" ).agg( "sum" )

cevt_atr_trn = cevt_atr_trn.merge( cant_atr_evt_trn, how = "left", on = "ref_hash" ).fillna( 0 )
cevt_atr_tst = cevt_atr_tst.merge( cant_atr_evt_tst, how = "left", on = "ref_hash" ).fillna( 0 )

cevt_atr_trn.to_csv( loc_ftr + "\\cevt_wat_trn.csv", index = False )
cevt_atr_tst.to_csv( loc_ftr + "\\cevt_wat_tst.csv", index = False )

## <span style="color:green"> **App mas eventeada con wifi** </span>

#### <span style='color:orange'> **Mean encoding** </span>

In [39]:
mapp_evt_trn = rh_trn.copy()
mapp_evt_tst = rh_tst.copy()

app_mas_evt_trn = evt_trn.loc[evt_trn['wifi'] == True].groupby( by = ["ref_hash","application_id"] ).agg( {"application_id":"count"} ).rename( columns = {"application_id":"cant_evt"} ).reset_index()
app_mas_evt_tst = evt_tst.loc[evt_tst['wifi'] == True].groupby( by = ["ref_hash","application_id"] ).agg( {"application_id":"count"} ).rename( columns = {"application_id":"cant_evt"} ).reset_index()

app_mas_evt_trn = app_mas_evt_trn.sort_values( by = ["ref_hash", "cant_evt"], ascending = True ).drop_duplicates( subset = "ref_hash", keep = "last" ); del( app_mas_evt_trn["cant_evt"] )
app_mas_evt_tst = app_mas_evt_tst.sort_values( by = ["ref_hash", "cant_evt"], ascending = True ).drop_duplicates( subset = "ref_hash", keep = "last" ); del( app_mas_evt_tst["cant_evt"] ) 

mapp_evt_trn = mapp_evt_trn.merge( app_mas_evt_trn, how = "left", on = "ref_hash" )
mapp_evt_tst = mapp_evt_tst.merge( app_mas_evt_tst, how = "left", on = "ref_hash" )

mapp_evt_trn["to_count"] = 1; mapp_evt_trn["application_id"] = mapp_evt_trn[["application_id", "to_count"]].groupby("application_id").transform( "sum" ) / len(mapp_evt_trn); del(mapp_evt_trn["to_count"])
mapp_evt_tst["to_count"] = 1; mapp_evt_tst["application_id"] = mapp_evt_tst[["application_id", "to_count"]].groupby("application_id").transform( "sum" ) / len(mapp_evt_tst); del(mapp_evt_tst["to_count"])

#Completamos los NaNs con el promedio de su cantidad
mapp_evt_trn["application_id"].fillna( mapp_evt_trn["application_id"].isnull().sum() / len( mapp_evt_trn ) , inplace = True )
mapp_evt_tst["application_id"].fillna( mapp_evt_tst["application_id"].isnull().sum() / len( mapp_evt_tst ), inplace = True )

mapp_evt_trn.to_csv( loc_ftr + "\\mapw_evt_trn.csv", index = False )
mapp_evt_tst.to_csv( loc_ftr + "\\mapw_evt_tst.csv", index = False )


## <span style="color:green"> **Cantidad eventos con Wifi en App mas eventeada por Usuario** </span>

In [40]:
mapp_evt_trn = rh_trn.copy()
mapp_evt_tst = rh_tst.copy()

app_mas_evt_trn = evt_trn.loc[evt_trn['wifi'] == True].groupby( by = ["ref_hash"] ).agg( {"application_id":"count"} ).rename( columns = {"application_id":"cant_evt"} ).reset_index()
app_mas_evt_tst = evt_tst.loc[evt_tst['wifi'] == True].groupby( by = ["ref_hash"] ).agg( {"application_id":"count"} ).rename( columns = {"application_id":"cant_evt"} ).reset_index()

app_mas_evt_trn = app_mas_evt_trn.sort_values( by = ["ref_hash", "cant_evt"], ascending = True ).drop_duplicates( subset = "ref_hash", keep = "last" );
app_mas_evt_tst = app_mas_evt_tst.sort_values( by = ["ref_hash", "cant_evt"], ascending = True ).drop_duplicates( subset = "ref_hash", keep = "last" );

mapp_evt_trn = mapp_evt_trn.merge( app_mas_evt_trn, how = "left", on = "ref_hash" )
mapp_evt_tst = mapp_evt_tst.merge( app_mas_evt_tst, how = "left", on = "ref_hash" )

mapp_evt_trn["cant_evt"].fillna( 0.0 , inplace = True )
mapp_evt_tst["cant_evt"].fillna( 0.0 , inplace = True )

mapp_evt_trn.to_csv( loc_ftr + "\\cmapw_ev_trn.csv", index = False )
mapp_evt_tst.to_csv( loc_ftr + "\\cmapw_ev_tst.csv", index = False )